In [ ]:
# ==================== PREDICTION CONFIDENCE ANALYSIS ====================
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from transformers import ViTImageProcessor, ViTForImageClassification
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==================== SETUP ====================
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")

def vit_transformers(image):
    inputs = processor(images=image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

test_dataset = ImageFolder(
    root=r"C:\Users\Ahmed Pasha\OneDrive\Desktop\garbage\data\test",
    transform=vit_transformers
)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
class_names = test_dataset.classes

# Load model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(class_names),
    ignore_mismatched_sizes=True
)
model.load_state_dict(torch.load("C:\Users\Ahmed Pasha\OneDrive\Desktop\garbage\model\best_vit_model.pth", map_location=device))
model.to(device)
model.eval()

# ==================== GET DETAILED PREDICTIONS ====================
def get_detailed_predictions(model, loader, dataset):
    all_predictions = []
    all_labels = []
    all_confidences = []
    all_image_paths = []
    
    idx = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(pixel_values=images)
            probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
            confidences, predictions = torch.max(probabilities, 1)
            
            batch_size = images.size(0)
            for i in range(batch_size):
                all_predictions.append(predictions[i].item())
                all_labels.append(labels[i].item())
                all_confidences.append(confidences[i].item())
                all_image_paths.append(dataset.imgs[idx][0])
                idx += 1
    
    return all_predictions, all_labels, all_confidences, all_image_paths

print("🔄 Getting detailed predictions...")
predictions, labels, confidences, image_paths = get_detailed_predictions(
    model, test_loader, test_dataset
)
print("✅ Predictions obtained!")

# Convert to numpy arrays
predictions = np.array(predictions)
labels = np.array(labels)
confidences = np.array(confidences)

# Calculate correctness
correct = predictions == labels

# ==================== CONFIDENCE STATISTICS ====================
print("\n" + "="*80)
print("CONFIDENCE STATISTICS")
print("="*80)
print(f"Mean Confidence: {np.mean(confidences):.4f}")
print(f"Median Confidence: {np.median(confidences):.4f}")
print(f"Std Confidence: {np.std(confidences):.4f}")
print(f"Min Confidence: {np.min(confidences):.4f}")
print(f"Max Confidence: {np.max(confidences):.4f}")

print("\nCorrect vs Incorrect Predictions:")
print(f"Mean Confidence (Correct): {np.mean(confidences[correct]):.4f}")
print(f"Mean Confidence (Incorrect): {np.mean(confidences[~correct]):.4f}")

# ==================== CONFIDENCE DISTRIBUTION ====================
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(confidences[correct], bins=50, alpha=0.7, label='Correct', color='green', edgecolor='black')
plt.hist(confidences[~correct], bins=50, alpha=0.7, label='Incorrect', color='red', edgecolor='black')
plt.xlabel('Confidence', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Confidence Distribution', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot([confidences[correct], confidences[~correct]], 
            labels=['Correct', 'Incorrect'],
            patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='black'),
            medianprops=dict(color='red', linewidth=2))
plt.ylabel('Confidence', fontsize=12)
plt.title('Confidence Box Plot', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('confidence_distribution.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Confidence distribution saved to 'confidence_distribution.png'")

# ==================== CONFIDENCE BY CLASS ====================
confidence_by_class = {class_name: [] for class_name in class_names}
for i in range(len(predictions)):
    pred_class = class_names[predictions[i]]
    confidence_by_class[pred_class].append(confidences[i])

mean_confidence_by_class = {k: np.mean(v) if v else 0 for k, v in confidence_by_class.items()}

plt.figure(figsize=(12, 6))
plt.bar(mean_confidence_by_class.keys(), mean_confidence_by_class.values(), 
        color='steelblue', edgecolor='black')
plt.xlabel('Class', fontsize=12)
plt.ylabel('Mean Confidence', fontsize=12)
plt.title('Mean Prediction Confidence by Class', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.ylim([0, 1.1])
for i, (k, v) in enumerate(mean_confidence_by_class.items()):
    plt.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom')
plt.tight_layout()
plt.savefig('confidence_by_class.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Confidence by class saved to 'confidence_by_class.png'")

# ==================== LOW CONFIDENCE

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3605151825.py, line 34)